In [1]:
import sqlite3
import pandas as pd

In [2]:
#Conectar a base de datos
conn = sqlite3.connect("database.db")
cursor= conn.cursor()

In [3]:
def sql_query(query):
    return pd.read_sql(query, conn)

In [4]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS Cervezas(
    CodC TEXT PRIMARY KEY,
    Envase TEXT NOT NULL,
    Capacidad REAL NOT NULL,
    Stock INTEGER NOT NULL
)
''')

valores=[
    ("01","Botella", 0.2,3600),
    ("02","Botella", 0.33,1200),
    ("03","Lata", 0.33,2400),
    ("04","Botella",1,288),
    ("05","Barril",60,30),
]

#Insertar datos
cursor.executemany("""
INSERT OR IGNORE INTO Cervezas (CodC,Envase,Capacidad,Stock)
VALUES(?,?,?,?)
""",valores)

conn.commit()

In [5]:
cursor.execute ('''
CREATE TABLE IF NOT EXISTS Bares(
    CodB TEXT PRIMARY KEY,
    Nombre TEXT NOT NULL,
    Cif TEXT,
    Localidad TEXT NOT NULL
)
''')

valores=[
    ("001","Stop","11111111X","Ville Botijo"),
    ("002","Las Vegas","22222222Y","Ville Botijo"),
    ("003","Club Social",None,"Las Ranas"),
    ("004","Otra Ronda","33333333Z","La Esponja"),
]

#Insertar datos
cursor.executemany("""
INSERT OR IGNORE INTO Bares (CodB,Nombre,Cif,Localidad)
VALUES(?,?,?,?)
""",valores)

conn.commit()

In [6]:
cursor.execute ('''
CREATE TABLE IF NOT EXISTS Empleados(
    CodE INTEGER PRIMARY KEY,
    Nombre TEXT NOT NULL,
    Sueldo INTEGER NOT NULL
)
''')

valores=[
    (1,"Carlos Lopez",120000),
    (2,"Rosa Perez",110000),
    (3,"Luisa García",100000),
    
]

#Insertar datos
cursor.executemany("""
INSERT OR IGNORE INTO Empleados (CodE,Nombre,Sueldo)
VALUES(?,?,?)
""",valores)

conn.commit()

In [7]:
#Tabla reparto
cursor.execute("""
CREATE TABLE IF NOT EXISTS Reparto (
    CodE INTEGER,
    CodB INTEGER,
    CodC INTEGER,
    Fecha DATE,
    Cantidad INTEGER,
    PRIMARY KEY(CodE,CodB,CodC,Fecha),
    FOREIGN KEY(CodE) REFERENCES Empleados(CodE)
    FOREIGN KEY(CodB) REFERENCES Bares(CodC)
    FOREIGN KEY(CodC) REFERENCES Cervezas(CodC)
)
""")

valores=[
    (1,'001','01','10-21-05',240),
    (1,'001','02','10-21-05',48),
    (1,'002','03','10-22-05',60),
    (1,'004','05','10-22-05',4),
    (2,'002','03','10-22-05',48),
    (2,'002','05','10-23-05',2),
    (2,'004','01','10-23-05',480),
    (2,'004','02','10-24-05',72),
    (3,'003','03','10-24-05',48),
    (3,'003','04','10-25-05',20),
]

#Insertar datos
cursor.executemany("""
INSERT OR IGNORE INTO Reparto (CodE,CodB,CodC,Fecha,Cantidad)
VALUES(?,?,?,?,?)
""",valores)

conn.commit()



In [8]:
query = '''
SELECT *
FROM Cervezas
'''
df = sql_query(query)
df

,CodC,Envase,Capacidad,Stock
0,1,Botella,0.20,3600
1,2,Botella,0.33,1200
2,3,Lata,0.33,2400
3,4,Botella,1.00,288
4,5,Barril,60.00,30


In [9]:
query = '''
SELECT *
FROM Bares
'''
df = sql_query(query)
df

,CodB,Nombre,Cif,Localidad
0,001,Stop,11111111X,Ville Botijo
1,002,Las Vegas,22222222Y,Ville Botijo
2,003,Club Social,None,Las Ranas
3,004,Otra Ronda,33333333Z,La Esponja


In [10]:
query = '''
SELECT *
FROM Empleados
'''
df = sql_query(query)
df

,CodE,Nombre,Sueldo
0,1,Carlos Lopez,120000
1,2,Rosa Perez,110000
2,3,Luisa García,100000


In [11]:
query = '''
SELECT *
FROM Reparto
'''
df = sql_query(query)
df

,CodE,CodB,CodC,Fecha,Cantidad
0,1,1,1,10-21-05,240
1,1,1,2,10-21-05,48
2,1,2,3,10-22-05,60
3,1,4,5,10-22-05,4
4,2,2,3,10-22-05,48
5,2,2,5,10-23-05,2
6,2,4,1,10-23-05,480
7,2,4,2,10-24-05,72
8,3,3,3,10-24-05,48
9,3,3,4,10-25-05,20


In [12]:
# 1.Obtener  el  nombre    de  los  empleados   que  hayan  repartido  al  bar  Stop  durante la semana del 17 al 23 de octubre de 2005. 
query = '''
SELECT DISTINCT E.Nombre, R.CodB, R.Fecha
FROM Reparto as R
JOIN Empleados as E ON R.CodE = E.CodE
WHERE R.CodB = '001'
  AND R.Fecha BETWEEN '10-17-05' AND '10-23-05'
'''

df = sql_query(query)
df


,Nombre,CodB,Fecha
0,Carlos Lopez,1,10-21-05


In [13]:
#2. Obtener  el   Cif  y  nombre  de  los  bares  a  los  que  se  ha  repartido  cerveza  de  tipo  Botella  y  capacidad inferior a 1 litro, ordenados por localidad
query = '''
SELECT DISTINCT B.Cif, B.Nombre
FROM Reparto as R
JOIN Cervezas as C ON R.CodC = C.CodC
JOIN Bares as B ON R.CodB = B.CodB
WHERE C.Envase = 'Botella' AND C.Capacidad < 1
ORDER BY B.Localidad
'''

df = sql_query(query)
df

,Cif,Nombre
0,33333333Z,Otra Ronda
1,11111111X,Stop


In [14]:
#3. Obtener los repartos (nombre del bar, envase y capacidad de la bebida, fecha y cantidad) realizados por Carlos Lopez.

query = '''
SELECT DISTINCT B.Nombre, C.Envase, C.Capacidad, R.Fecha, R.Cantidad
FROM Reparto as R
JOIN Cervezas as C ON R.CodC = C.CodC
JOIN Bares as B ON R.CodB = B.CodB
WHERE R.CodE = 1
'''

df = sql_query(query)
df


,Nombre,Envase,Capacidad,Fecha,Cantidad
0,Stop,Botella,0.20,10-21-05,240
1,Stop,Botella,0.33,10-21-05,48
2,Las Vegas,Lata,0.33,10-22-05,60
3,Otra Ronda,Barril,60.00,10-22-05,4


In [15]:
#4. Obtener los bares a los que se les ha repartido envases de tipo botella y capacidad 0.2 ó0.33

query = '''
SELECT DISTINCT B.Nombre, R.Fecha
FROM Reparto as R
JOIN Cervezas as C ON R.CodC = C.CodC
JOIN Bares as B ON R.CodB = B.CodB
WHERE (C.Envase = "Botella") AND (C.Capacidad IN (0.2,0.33))
'''

df = sql_query(query)
df


,Nombre,Fecha
0,Stop,10-21-05
1,Otra Ronda,10-23-05
2,Otra Ronda,10-24-05


In [23]:
#5.Nombre de los empleados que han repartido a los bares "Stop" y "Las Vegas" cervezas conenvase botella.

query = '''
SELECT DISTINCT B.Nombre, E.Nombre, R.Fecha
FROM Reparto as R
JOIN Cervezas as C ON R.CodC = C.CodC
JOIN Bares as B ON R.CodB = B.CodB
JOIN Empleados as E ON R.CodE = E.CodE
WHERE (C.Envase = "Botella") AND (B.Nombre IN ("Stop","Las Vegas"));
'''

df = sql_query(query)
df


,Nombre,Nombre,Fecha
0,Stop,Carlos Lopez,10-21-05


In [17]:
#6. Obtener el nombre y número de viajes que ha realizado cada empleado fuera de Villa Botijo.

query = '''
SELECT E.Nombre, COUNT(*) as NumeroViajes
FROM Reparto R
JOIN Bares B ON R.CodB = B.CodB
JOIN Empleados E ON R.CodE = E.CodE
WHERE B.Localidad != 'Ville Botijo'
GROUP BY E.Nombre
'''

df = sql_query(query)
df

,Nombre,NumeroViajes
0,Carlos Lopez,1
1,Luisa García,2
2,Rosa Perez,2


In [18]:
#7. Obtener el nombre y localidad del bar que más litros de cerveza ha comprado.
#B.Nombre B.Localidad
query = '''
SELECT B.Nombre, B.Localidad
FROM Bares as B
JOIN Reparto R ON B.CodB = R.CodB
JOIN Cervezas C ON R.CodC = C.CodC
GROUP BY B.CodB
ORDER BY SUM(R.Cantidad * C.Capacidad) DESC
LIMIT 1;
'''

df = sql_query(query)
df

,Nombre,Localidad
0,Otra Ronda,La Esponja


In [19]:
#8. Obtener los bares que han adquirido todos los tipos de cerveza con envase de botella y capacidad menor que 1 litro

query = '''
SELECT DISTINCT B.Nombre
FROM Reparto R
JOIN Bares as B ON R.CodB = B.CodB
JOIN Cervezas as C ON R.CodC = C.CodC
WHERE (C.Envase = "Botella") AND (Capacidad < 1)
'''

df = sql_query(query)
df


,Nombre
0,Stop
1,Otra Ronda


In [26]:
# 9. Subir un 5% el sueldo del empleado que más días haya trabajado.
query = '''
SELECT CodE
FROM Reparto
GROUP BY CodE
ORDER BY COUNT(DISTINCT Fecha) DESC
LIMIT 1;
'''

df = sql_query(query)
df


,CodE
0,2
